In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_json('script-bag-of-words.json')

In [4]:
df.head()

,episodeAlt,seasonNum,episodeNum,episodeTitle,text
0,S1E1,1,1,Winter Is Coming,"[{'text': 'Easy, boy.', 'name': 'Will'}, {'tex..."
1,S1E2,1,2,The Kingsroad,"[{'text': 'And You child... drink, eat. need t..."
2,S1E3,1,3,Lord Snow,[{'text': 'Council. Grand Lord Maester Pycelle...
3,S1E4,1,4,"Cripples, Bastards, and Broken Things",[{'text': 'Lord's The again. been dreaming lit...
4,S1E5,1,5,The Wolf and the Lion,[{'text': 'Does Hugh Ser any capital? family h...


In [28]:
dialogue = {}

for _, row in df.iterrows():
    for item in row['text']:
        name = item['name']
        text = item['text'].strip()

        if len(text) == 0:
            continue

        if name in dialogue:
            dialogue[name] += " " + text
        else:
            dialogue[name] = text


In [29]:
len(dialouge)

817

In [30]:
new_df=pd.DataFrame()
new_df['character']=dialouge.keys()
new_df['words']=dialouge.values()

In [31]:
new_df['num_words']=new_df['words'].apply(lambda x:len(x.split()))

In [32]:
new_df=new_df.sort_values('num_words',ascending=True)

In [33]:
new_df=new_df.head(100)

In [34]:
new_df.shape

(100, 3)

In [35]:
new_df.head()

,character,words,num_words
432,Meereenese Man,,0
402,Manservant,,0
457,Son of the Harpy,,0
391,Great Master #1,,0
525,Pit Announcer,,0


In [36]:
new_df

,character,words,num_words
432,Meereenese Man,,0
402,Manservant,,0
457,Son of the Harpy,,0
391,Great Master #1,,0
525,Pit Announcer,,0
...,...,...,...
346,Night's Watch Guard,Horse approaching!,2
385,Farm Man,The Wildlings!,2
244,Stark Man #4,Hang him!,2
271,Wildling Boy,Crow! Crow!Crow!,2


In [42]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(
    stop_words='english',
    min_df=2    
)


In [38]:
emb = cv.fit_transform(new_df['words'])


In [40]:
from sklearn.manifold import TSNE

z = TSNE(
    n_components=2,
    random_state=123,
    perplexity=30
).fit_transform(emb.toarray())


In [43]:
z.shape

(100, 2)

In [44]:
new_df['x']=z.T[0]
new_df['y']=z.T[1]

In [45]:
new_df

,character,words,num_words,x,y
432,Meereenese Man,,0,-253.227737,-711.005493
402,Manservant,,0,616.027832,81.526344
457,Son of the Harpy,,0,-528.728516,-589.565063
391,Great Master #1,,0,-766.850586,-354.021973
525,Pit Announcer,,0,343.979706,-464.045990
...,...,...,...,...,...
346,Night's Watch Guard,Horse approaching!,2,-109.824089,38.763680
385,Farm Man,The Wildlings!,2,-109.824089,38.763680
244,Stark Man #4,Hang him!,2,-109.824089,38.763680
271,Wildling Boy,Crow! Crow!Crow!,2,-109.824089,38.763680


In [46]:
new_df

,character,words,num_words,x,y
432,Meereenese Man,,0,-253.227737,-711.005493
402,Manservant,,0,616.027832,81.526344
457,Son of the Harpy,,0,-528.728516,-589.565063
391,Great Master #1,,0,-766.850586,-354.021973
525,Pit Announcer,,0,343.979706,-464.045990
...,...,...,...,...,...
346,Night's Watch Guard,Horse approaching!,2,-109.824089,38.763680
385,Farm Man,The Wildlings!,2,-109.824089,38.763680
244,Stark Man #4,Hang him!,2,-109.824089,38.763680
271,Wildling Boy,Crow! Crow!Crow!,2,-109.824089,38.763680


In [47]:
final_df = new_df[['character', 'x', 'y']].copy()
final_df.head()


,character,x,y
432,Meereenese Man,-253.227737,-711.005493
402,Manservant,616.027832,81.526344
457,Son of the Harpy,-528.728516,-589.565063
391,Great Master #1,-766.850586,-354.021973
525,Pit Announcer,343.979706,-464.045990


In [48]:
import pickle

with open("data.pkl", "wb") as f:
    pickle.dump(final_df, f)
